### Document Clustering 2

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
import konlpy.tag 
import pandas as pd
import scipy as sp
import itertools
import operator

##### 1. Make Three Texts

In [10]:
text1 = """
국민의당의 박지원 비대위원장 체제가 본격 가동됐습니다.
안철수·천정배 두 공동대표의 사퇴로 임시지도부를 이끌게 된 박지원 비상대책위원장은 '마냥 슬퍼할 수만은 없다'며 당 정상화 의지를 강조했습니다.
[박지원/국민의당 비상대책위원장]
"바쁜 꿀벌은 슬퍼할 시간이 없습니다. 위기를 극복하는 정답은 일하는 것입니다."
동교동계 출신의 최경환 의원을 비대위원장 비서실장으로 임명하며 구체적인 당 수습 방안도 내놨습니다.
최대한 빨리 비대위원 인선을 마치고 인사는 최소화해 당직자들의 동요를 막기로 했습니다.
새 지도부를 선출할 전당대회 시기는 당 수습부터 마친 후 결정하기로 했고, 매주 화요일 의원총회를 정례화하기로 했습니다.
7~8월에는 안철수·천정배 전 대표를 포함한 의원들이 전국 순회 간담회, '민생 투어'로 실추된 이미지 회복에 나설 계획도 밝혔습니다.
이번 사태를 가져온 박선숙·김수민 의원과 안철수 전 대표는 의원총회에 불참했습니다.
안 전 대표는 마지막 아침 공부모임에는 참석해 '개근상'을 받은 후 앞으로 백의종군하겠다고 밝혔습니다.
"""

In [11]:
text2 = """
여야의 ‘불체포 특권’ 내려놓기가 가시화되는 분위기다. 친인척 보좌진 채용 논란으로 인한 사회적 질타를 극복하기 위한 타개책의 일환으로 인식된다.
국회의장과 여야 3당 원내대표 간 30일 만찬회동에서 정세균 의장은 자신의 취임 공약인 ‘국회의원 특권 내려놓기’ 자문기구 설치를 제안했다. 정진석 새누리당, 우상호 더불어민주당, 박지원 국민의당 원내대표는 이견 없이 흔쾌히 합의했다. 최근 국회가 ‘갑질’과 ‘특권’의 대명사로 불리며 사회적 지탄 대상으로 떠오르다보니 속도감 있게 합의안을 도출한 것으로 보인다.
정 원내대표는 회동이 끝난 뒤 “20대 국회를 생산적으로 잘 좀 이끌어보자. 대화와 타협을 통해 제대로 협치를 이뤄보자고 덕담을 나눴다”고 말했다.
그러나 여론의 영향이 덜하고 각 당의 정치적 셈법과 관련된 부분에 있어서는 접점을 찾지 못한 것으로 전해졌다. 정 의장은 국회 개헌특별위원회 설치를 제안했지만 새누리당의 반대로 합의가 이뤄지지 못한 것으로 알려졌다. 이에 따라 국회의장 직속 개헌 자문기구를 구성하는 수준에서 개헌 논의가 시작될 것으로 예상된다.
이밖에 정부와 여당이 요구하는 서비스산업발전기본법과 노동개혁법 처리와 야당이 요구하는 세월호 특조위 활동 기간 연장 등 문제를 놓고선 견해차를 좁히지 못한 것으로 전해졌다. 때문에 여론에 민감한 정치인들의 속성을 그대로 보여준 합의에 불과한 게 아니냐는 비판도 제기된다.
한편 새누리당은 이날 국회의원의 불체포특권 폐지와 세비 동결을 포함한 고강도 ‘국회 개혁안’을 내놓았다. 새누리당 혁신비상대책위원회가 이날 의결한 개혁안의 핵심은 불체포특권 폐지다. 의원이 범죄 혐의가 있을 때마다 논란이 됐던 ‘방탄 국회’ 시비를 차단하겠다는 뜻이 담겨 있다.
또 의원 징계의 실효성을 높이기 위해 징계안은 60일 이내에 반드시 심의하고 이를 넘길 경우 본회의에 징계안을 곧바로 상정할 수 있도록 국회법을 개정하기로 했다. 윤리특위 산하 민간기구인 윤리심사자문위를 ‘윤리심사위’로 바꿔 권한과 역할을 강화하기로 했다. 윤리특위가 ‘제 식구 감싸기’ 식으로 운영된다는 비판을 의식한 것으로 해석된다.
이와 함께 비대위는 20대 국회 세비 동결을 결의했고 민간위원회를 구성해 본회의와 상임위 등의 출석수당도 전면 손질하기로 했다. 올해 소속 의원 전원이 100만원 이상의 성금을 갹출해 ‘청년희망펀드’ 등에 기부하도록 결의했다.
"""

In [12]:
text3 = """
여야(與野) 3당이 '국회의원 특권 내려놓기' 방안을 앞다퉈 쏟아내고 있다. 여야뿐 아니라 국회의장까지 의원 특권 내려놓기를 추진하는 등 국회의원 특권 타파가 정치권의 중심 이슈가 되고 있다. 여야는 과거에도 수차례 특권을 내려놓겠다고 공언(公言)했지만 이런저런 이유를 대면서 '공언(空言)'으로 만들었다.
새누리당은 30일 국회의원 불체포 특권 등 의원 특권을 대폭 내려놓겠다고 했다. 이를 위해 의원 체포 동의안을 국회 본회의 보고 72시간 안에 처리하지 못하면 자동 폐기되는 국회법 규정을 없애겠다고 했다. 72시간 내 표결이 이뤄지지 않으면 이후 열리는 첫 본회의에 체포안을 자동 상정하는 조항을 신설하는 국회법 개정을 추진한다는 것이다. 불체포 특권은 '국회 동의 없이 의원을 회기(會期) 중엔 체포할 수 없다'는 헌법상 권리지만, 범죄 혐의를 받는 의원들이 법망을 피하는 수단으로 악용됐다는 지적이다. 새누리당은 이와 함께 △의원 영장실질심사 자진 출석 의무화 △국회 세비(歲費) 동결 △8촌 이내 친·인척 보좌진 채용 금지 △보좌진으로부터 후원금 징수 금지 등을 위한 관련법 개정도 추진하기로 했다.
더불어민주당은 소속 의원들이 법안 발의를 통해 특권 내려놓기 추진에 나섰다. 원혜영 의원은 최근 '국회 본회의 등에 4분의 1 이상 무단결석할 경우 회의수당 전액을 삭감'하는 내용의 '국회의원수당법' 개정안을 발의했다. 백혜련 의원은 보좌진 보수를 유용하면 형사처벌하는 이른바 '국회의원 갑질 금지 법안'을 발의했다. 백재현 국회 윤리특별위원장은 이른바 '국회의원 금배지' 폐지를 추진하겠다고 밝혔다. 국민의당도 지난 총선에서 정치인 낙하산 임명 금지, 국회의원 국민소환제(파면제) 등을 공약한 데 이어 안철수 전 대표가 지난 22일 국회 대표 연설에서 의원 특권 내려놓기 추진을 위한 국회 차원 기구 구성을 제안했다.
이런 가운데 정세균 국회의장은 이날 여야 3당 원내대표와 만나 '국회의원 특권 내려놓기' 관련 의장 자문기구를 설치하기로 합의했다.
하지만 정치권이 경쟁적으로 내놓는 특권 내려놓기 방안은 새로운 게 아니다. 역대 국회에서 개원(開院) 직후나 총선·대선 등 주요 선거를 앞두고 단골 공약으로 등장했다가 국회 임기 만료와 함께 슬그머니 사라진 방안들이다. 이 때문에 "매번 일이 터지면 특권을 내려놓겠다고 했지만 제대로 법제화하거나 실천한 적이 없다"는 비판을 받아왔다.
이번에도 정치권이 말만 요란한 경쟁에 나선 것 아니냐는 지적이 벌써 나온다.
여야가 20대 국회 출범과 함께 특권 내려놓기에 열을 올리는 것은 내년 대선을 염두에 두고 벌이는 '혁신(革新)' 경쟁의 측면도 있다. 이런 가운데 최근 더민주 서영교 의원의 '가족 보좌진 채용' 논란과 국민의당 김수민 의원 홍보비 리베이트 논란이 터지면서 정치권에 대한 불신이 커졌다. 새누리당 역시 이군현 의원의 보좌진 월급 상납 의혹 사건 등 갑질 논란에서 자유롭지 못한 상황이다.
여야 3당 대표들은 최근 국회 연설에서 공히 한국 사회가 처한 위기의 핵심을 '양극화 심화'로 보고 '격차 해소'를 목표로 내세웠다. 내년 대선에서 격차 해소와 특권 타파가 쟁점으로 떠오를 가능성이 큰 만큼 정치권이 여론을 의식해 자신들의 특권 내려놓기를 공약하고 나섰다는 분석이다. 문제는 역대 국회가 약속해온 특권 포기 공약 중 법제화와 실천으로 이어진 경우는 드물었다는 점이다.
새누리당이 이날 추진하기로 한 방안 중 대부분은 지난해 당 보수혁신특별위원회가 이미 마련해 공개했던 내용이다. 법제화된 내용은 없다. 불체포 특권 제한 법안은 지난해 1월 당론 발의됐으나 상임위에서 제대로 논의되지 못하고 19대 국회가 종료되면서 폐기됐다.
여야는 19대 국회 때 "방탄 국회는 없다"고 공언했지만 2014년 9월 새누리당 송광호 의원에 대한 체포 동의안을 부결시켰다. 당시 재적 의원 223명 가운데 118명이 반대표를 던졌다. 여당뿐 아니라 야당 의원들도 체포 동의안 부결에 합세한 것이다. 겉으론 특권을 내려놓겠다면서 뒤에선 특권 유지를 위한 여야 간 담합(談合)이 작동했다는 지적이 나오는 이유다.
매번 공약으로 등장하지만 지켜지지 않는 이유에 대해 정치권에서는 "국회의원들이 특권을 내려놓을 의지가 없기 때문"이란 말이 나온다. 한 초선 의원은 "친·인척 보좌진 채용을 금지하면 의원들끼리 서로 가족 보좌진을 주고받을 게 뻔하고, 후원금 역시 마찬가지 아니겠느냐"며 "제도가 아니라 의원들의 의지가 중요하다"고 했다.
"""

##### 2. Analysis Morphological & Documentation

In [18]:
def make_document_morpheme(texts):
    """make documents list after morphological"""
    
    documents = []
    
    komoran = konlpy.tag.Komoran()
    for text in texts:
        obj = line2obj(komoran.pos(text, flatten=False))
        documents.append(obj)
        
    return documents

def line2obj(lines):
    """ make keywords dictionary include only (NNP, NNG)"""
    
    obj = {}
    
    for line in lines:
        for keyword in line:
            if len(keyword[1]) > 1 and ( keyword[1] == "NNP" or keyword[1] == "NNG" ) :
                
                key = keyword[0]
                if key in obj:
                    obj[key] += 1
                else:
                    obj[key] = 1
                    
    return obj

In [19]:
texts = [text1, text2, text3]
documents = make_document_morpheme(texts)

In [27]:
def gen_text_documents(documents):
    
    text_documents = []
    
    for document in documents:

        text_words = []
        
        for word, count in document.items():
            text_words.extend([word] * count)
            
        text_document = " ".join(text_words)
        text_documents.append(text_document)
        
    return text_documents    

In [30]:
text_documents = gen_text_documents(documents)
text_documents[0]

'대표 대표 대표 시간 국민의당 국민의당 김수미 아침 비서실장 의지 체제 위기 참석 포함 모임 전당 총회 총회 개근상 가동 민생 방안 시기 인사 정상 위원 간담회 전국 안철수 안철수 안철수 임명 극복 공동대표 실추 당 당 당 출신 계획 당직자 천정배 천정배 최소 비대 비대 비대 마지막 수습 수습 동교동계 사퇴 임시 사태 정례 불참 대회 박선숙 최경환 투어 비상대책위 비상대책위 원장 원장 구체 최대한 공부 정답 이번 위원장 위원장 선출 이미지 후 후 회복 지도부 지도부 의원 의원 의원 의원 의원 순회 강조 결정 화요일 앞 인선 꿀벌 일 동요 백의종군 박지원 박지원 박지원'

##### 3. Count Vectorizer

In [43]:
vectorizer = CountVectorizer(min_df=1)
vectorized = vectorizer.fit_transform(text_documents)
num_docs, num_features =  vectorized.shape
feature_names = vectorizer.get_feature_names()

print("the number of input documents : {}".format(num_docs))
print("the number of words : {}".format(num_features))
print("feature word list : {} ...".format(feature_names[:5]))

the number of input documents : 3
the number of words : 323
feature word list : ['2014년', '9월', '가동', '가시', '가운데'] ...


In [63]:
print("word vector matrics")
print(vectorized.toarray())

word vector matrics
[[ 0  0  1  0  0  0  1  0  1  0  1  0  0  0  0  0  0  0  0  0  1  0  0  1
   0  1  1  0  0  0  0  0  1  0  2  0  0  0  0  0  0  0  1  0  0  0  0  0
   0  1  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  3  0  1  0
   0  0  1  1  0  0  0  0  1  0  0  0  1  0  0  0  0  1  0  1  3  0  0  0
   0  1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  3  0  2  1
   0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0
   0  0  0  0  0  2  0  0  1  1  1  0  0  0  0  0  0  1  0  0  0  0  0  1
   0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0
   0  1  1  2  0  0  0  0  0  0  5  0  1  0  0  0  0  0  1  1  0  0  0  0
   1  1  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  1
   1  0  1  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0
   0  1  0  0  0  2  0  1  0  0  0  2  1  0  1  1  0  0  0  1  0  0  0  0
   0  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0
   0  0  0  1  0  

In [64]:
pd.DataFrame(vectorized.toarray(), columns=feature_names)

,2014년,9월,가동,가시,가운데,가족,간담회,강도,강조,강화,...,형사,홍보비,화요일,활동,회기,회동,회복,회의,후원금,희망
0,0,0,1,0,0,0,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,1,0,1,...,0,0,0,1,0,1,0,0,0,1
2,1,1,0,0,3,2,0,0,0,0,...,1,1,0,0,1,0,0,1,2,0


##### 4. Euclid Distance Measurement

In [101]:
def calc_euc_dist(v1, v2):
    delta = v1 - v2
    dist = sp.linalg.norm(delta.toarray())
    return dist

def get_euc_dists(vectorized, num_docs):
    
    dists = []
    
    for i, j in itertools.combinations(range(num_docs), 2):
        dist = calc_euc_dist(vectorized.getrow(i), vectorized.getrow(j))
        dists.append((i, j, dist))
        
    return dists

def print_dists(dists):
    
    for i, j, dist in sorted(dists, key=operator.itemgetter(2)):
        print("{}\t{}\t{}".format(i, j, dist))
    

In [91]:
texts = [text1, text2, text3]
documents = make_document_morpheme(texts)
text_documents = gen_text_documents(documents)

In [102]:
vectorizer = CountVectorizer(min_df=1)
vectorized = vectorizer.fit_transform(text_documents)
num_docs, num_features =  vectorized.shape

dists = get_euc_dists(vectorized, num_docs)
print_dists(dists)

0	1	21.330729007701542
1	2	37.094473981982816
0	2	41.14608122288197
